In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import Parameter
import torch.nn.functional as F
import math

from sklearn.datasets import fetch_20newsgroups
import numpy as np

import argparse
from types import SimpleNamespace

import random
random.seed(1234)

import pandas as pd
import matplotlib.pyplot as plt

from nltk.tokenize import RegexpTokenizer

import bsg, tools

In [3]:
# Get corpus: 20 news groups

newsgroups_train = fetch_20newsgroups(subset='train')

window = 5

vocabulary, tokenized_corpus, word2idx, idx2word = tools.tokenize_corpus(newsgroups_train.data)

indexed_corpus = tools.index_tokenized_corpus(tokenized_corpus, word2idx)

corpus_center_words, corpus_context_words = tools.get_corpus_centers_contexts(indexed_corpus, window)

unigram = tools.make_unigram_dict(corpus_center_words)

In [4]:
args_dict = {"vocab_size" : len(vocabulary), "window" : window, 
             "batch_size" : 200, "optimizer" : 80, "learning_rate" : 0.0005, 
             "momentum" : 0.99, "num_epoch" : 50, "init_mult" : 1, 
             "variance" : 0.995, "start" : True, "nogpu" : True, 
             "embedding_dim" : 300, "freeze" : False}

args = SimpleNamespace(**args_dict)

In [ ]:
model = bsg.BSG(args.window, unigram, args.vocab_size)
optimizer = torch.optim.Adam(model.parameters(), args.learning_rate, betas=(args.momentum, 0.999))
model = bsg.train(model, args, optimizer, corpus_center_words[0:10000], corpus_context_words[0:10000])

/Users/patrickding/TAMU/research/bsg_pytorch/bsg.py:111: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  loss_epoch += loss.data[0]    # add loss to loss_epoch


In [14]:
print(len(corpus_center_words))
print(len(corpus_center_words[0:10000].unique()))

2539447
526


In [17]:
idx2word(corpus_center_words[0])

TypeError: 'dict' object is not callable

In [28]:
foo, counts = np.unique(corpus_center_words[0:10000].tolist(), return_counts=True)

In [48]:
test_idx2realidx = {idx: realidx for (idx , realidx) in enumerate(foo)}
test_realidx2idx = {realidx: idx for (idx , realidx) in enumerate(foo)}

In [49]:
from sklearn.metrics.pairwise import cosine_similarity
cos_sim_matrix = cosine_similarity(model.type_means(torch.tensor(foo)).detach().numpy(), 
                                   model.type_means(torch.tensor(foo)).detach().numpy())
def n_closest_words(word, cos_sim_matrix, n):
    word_index = test_realidx2idx.get(word2idx.get(word))
    print("word_index is " + str(word_index))
    close_words_indices = np.argsort(cos_sim_matrix[word_index])[-n:]
    print("close_words_indices.shape" + str(close_words_indices.shape))
    return [idx2word.get(test_idx2realidx.get(j)) for j in close_words_indices]

In [53]:
n_closest_words("washington", cos_sim_matrix, 20)

word_index is 412
close_words_indices.shape(20,)


['question',
 'accidental',
 'addition',
 'made',
 'requested',
 'answer',
 'reduced',
 'there',
 'where',
 'gas',
 'shelley',
 'would',
 'functionality',
 'blank',
 'curious',
 'kuo',
 'believe',
 'doubt',
 'harvard',
 'washington']